In [31]:
!pip install mediapipe opencv-python
!pip install pygame

In [46]:
import cv2
import mediapipe
import numpy
import pygame
mp_drawing = mediapipe.solutions.drawing_utils  #drawing tools for drawing lines and such
mp_pose = mediapipe.solutions.pose  #pose estimation models

pygame.mixer.init()  #intialize pygame mixer

#load sound files
current_category = None
forward_sound = pygame.mixer.Sound("forward.mp3")
backward_sound = pygame.mixer.Sound("backward.mp3")


In [47]:
sound = pygame.mixer.Sound("forward.mp3")
sound.play()

<img src='https://www.researchgate.net/publication/361071987/figure/fig2/AS:11431281214731862@1703733107226/33-Landmarks-detected-on-the-human-body-using-MediaPipe.png' alt='Labelled Joints Image' width='500'>

In [50]:
def calc_angle(first,second,third):
    first = numpy.array(first)  #first joint's coords
    second = numpy.array(second)  #second joint's coords
    third = numpy.array(third)  #third joint's coords

    #radians = [arctan of the vector b to c] - [arctan of the vector a to b]
    #if doing shoulder, elbow, wrist then it would be
    #radians = arctan(elbow[y]-wrist[y], elbow[x]-wrist[x]) - actan([shoulder[y]-elbow[y], shoulder[x]-elbow[x])
    radians = numpy.arctan2(third[1]-second[1], third[0]-second[0]) - numpy.arctan2(first[1]-second[1], first[0]-second[0])
    angle = numpy.abs(radians*180.0/numpy.pi)  #turn radians into angle

    if angle > 180.0:
        angle = 360-angle
    return angle

In [54]:
def audio(angle):
    global current_category
    try:
        if angle > 95:
            new_category = 'forward'
        elif 60 < angle <= 95:
            new_category = None
        else:
            new_category = 'backward'

        # Only stop and play a new sound if the category has changed
        if new_category != current_category:
            pygame.mixer.stop()  # Stop any currently playing sound
            if new_category == 'forward':
                forward_sound.play()
            elif new_category == 'backward':
                backward_sound.play()

            current_category = new_category  # Update the current category
    except Exception as e:
        print(f"Error in audio function: {e}")


In [55]:
# Function to adjust brightness
def adjust_brightness(image, alpha, beta):
    # Clip the values to be in range [0, 255] to ensure valid pixel values
    return np.clip(alpha * image + beta, 0, 255).astype(np.uint8)

In [56]:
# Get video feed
camera = cv2.VideoCapture(0)
try:
    with mp_pose.Pose(min_detection_confidence=0.5, min_tracking_confidence=0.5) as pose:
        while camera.isOpened():
            returned, frame = camera.read()
            
            # Adjust brightness (alpha is the scaling factor, beta is the added brightness)
            frame = adjust_brightness(frame, alpha=1.5, beta=70)  # Increase alpha and/or beta for more brightness

            # Detect poses
            image = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
            image.flags.writeable = False

            results = pose.process(image)

            image.flags.writeable = True
            image = cv2.cvtColor(image, cv2.COLOR_RGB2BGR)

            try:
                landmarks = results.pose_landmarks.landmark

                # Get coordinates of the nose, left shoulder, and right shoulder
                nose = [landmarks[mp_pose.PoseLandmark.NOSE.value].x, landmarks[mp_pose.PoseLandmark.NOSE.value].y]
                left_shoulder = [landmarks[mp_pose.PoseLandmark.LEFT_SHOULDER.value].x, landmarks[mp_pose.PoseLandmark.LEFT_SHOULDER.value].y]
                right_shoulder = [landmarks[mp_pose.PoseLandmark.RIGHT_SHOULDER.value].x, landmarks[mp_pose.PoseLandmark.RIGHT_SHOULDER.value].y]

                # Calculate the angle
                angle = calc_angle(left_shoulder, nose, right_shoulder)

                #Play sound
                audio(angle)

                # Determine posture
                if angle > 95:
                    posture = "Leaning Forward"
                elif 60 < angle <= 95:
                    posture = "Upright"
                else:
                    posture = "Leaning Backward"

                # Print angle and posture
                coords = tuple(np.multiply(nose, [700, 550]).astype(int))
                cv2.putText(image, f'Angle: {int(angle)} {posture}', coords, cv2.FONT_HERSHEY_SIMPLEX, 0.5, (255, 255, 255), 2, cv2.LINE_AA)

            except Exception as e:
                pass

            # Render poses
            mp_drawing.draw_landmarks(image, results.pose_landmarks, mp_pose.POSE_CONNECTIONS)

            cv2.imshow("Camera Feed", image)

            if cv2.waitKey(10) & 0xFF == ord('q'):
                break

finally:
    camera.release()
    cv2.destroyAllWindows()
